In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS
import csv


In [3]:
url="https://www.tripadvisor.in/Hotels-g187147-Paris_Ile_de_France-Hotels.html"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
     'Referer': 'https://www.google.com/',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive'
}
data=requests.get(url,headers=headers)
print(data)
soup=BS(data.content,'html.parser')

<Response [200]>


In [18]:
links = []
base_url="https://www.tripadvisor.in"
for u in soup.findAll('a', {'class': 'BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS'}):
    # Extract the URL from the 'href' attribute
    link = u.get('href')
    final_url=base_url+ link
    links.append(final_url)

print(links)


['https://www.tripadvisor.in/Hotel_Review-g187147-d228694-Reviews-Hotel_Malte_Astotel-Paris_Ile_de_France.html', 'https://www.tripadvisor.in/Hotel_Review-g187147-d228694-Reviews-Hotel_Malte_Astotel-Paris_Ile_de_France.html#REVIEWS', 'https://www.tripadvisor.in/ShowUserReviews-g187147-d228694-r924034237-Hotel_Malte_Astotel-Paris_Ile_de_France.html', 'https://www.tripadvisor.in/Hotel_Review-g187147-d229968-Reviews-Hotel_Astra_Opera_Astotel-Paris_Ile_de_France.html', 'https://www.tripadvisor.in/Hotel_Review-g187147-d229968-Reviews-Hotel_Astra_Opera_Astotel-Paris_Ile_de_France.html', 'https://www.tripadvisor.in/Hotel_Review-g187147-d229968-Reviews-Hotel_Astra_Opera_Astotel-Paris_Ile_de_France.html#REVIEWS', 'https://www.tripadvisor.in/ShowUserReviews-g187147-d229968-r944408521-Hotel_Astra_Opera_Astotel-Paris_Ile_de_France.html', 'https://www.tripadvisor.in/Hotel_Review-g187147-d228757-Reviews-Hotel_Maison_Mere-Paris_Ile_de_France.html', 'https://www.tripadvisor.in/Hotel_Review-g187147-d228

In [20]:
print(len(links))

115


In [32]:
url1='https://www.tripadvisor.in/Hotel_Review-g187147-d228694-Reviews-Hotel_Malte_Astotel-Paris_Ile_de_France.html'
data1 = requests.get(url1,headers=headers)
soup1=BS(data1.content,'html.parser')
print(data1)

hotel_name1 = soup.find('h1', {'class': 'biGQs _P rRtyp'}).text.strip()
print(hotel_name1)



<Response [200]>
Hotel Malte Astotel


In [60]:
import re
import random
import time

Review_List = []

# Step 2: Scrape reviews from each hotel
for hotel_url in links[30:35]:  # Traverse only the first 5 hotel URLs
    print(f"Scraping reviews for hotel: {hotel_url}")
    
    for i in range(1, 15):  # Adjust the number of review pages as needed
        review_page_url = f"{hotel_url}-or{i * 10}"
        response = requests.get(review_page_url, headers=headers)
        soup = BS(response.content, 'html.parser')

        # Extract the full hotel name (no slicing)
        hname = soup.find('h1', {'class': 'biGQs _P rRtyp'})
        if hname and hname.text.strip():
            hotel_name = hname.text.strip()
        else:
            hotel_name = "Paris Hotel"

        # Extract reviewer names
        reviewer_name = []
        for rn in soup.findAll('a', {'class': 'blnum BDpWK'}):
            reviewer_name.append(rn.text.strip())

        # Extracting review content
        review = []
        for r in soup.findAll('span', {'class': 'JbGkU Cj'}):
            review.append(r.text.strip())

        # Extracting stay dates
        stay_date = []
        for d in soup.findAll('span', {'class': 'iSNGb _R Me S4 H3 Cj'}):
            sd = d.text.strip()
            match1 = re.search(r'\b[A-Za-z]+\s\d{4}\b', sd)
            if match1:
                stay_date.append(match1.group(0))

        # Extracting ratings
        stars = []
        for s in soup.findAll('div', {'class': 'IaVba F1'}):
            stars.append(s.text.strip())

        # Ensure all lists have the same length by truncating them to the minimum length
        min_length = min(len(reviewer_name), len(review), len(stay_date), len(stars))

        # Truncate only the lists (not the hotel name)
        reviewer_name = reviewer_name[:min_length]
        review = review[:min_length]
        stay_date = stay_date[:min_length]
        stars = stars[:min_length]

        # Append each complete review set to the Review_List
        for j in range(min_length):
            Review_List.append({
                "Hotel URL": hotel_url,
                "Hotel Name": hotel_name,  # Use the full hotel name directly without slicing
                "Reviewer Name": reviewer_name[j],
                "Review": review[j],
                "Stay Date": stay_date[j],
                "Stars": stars[j]
            })

        # Random sleep time to prevent blocking
        sleep_time = random.uniform(2, 5)  # Sleep for 2 to 5 seconds
        time.sleep(sleep_time)

# Step 3: Convert the Review_List to a DataFrame and save it
df = pd.DataFrame(Review_List)
print(df.head())



Scraping reviews for hotel: https://www.tripadvisor.in/ShowUserReviews-g187147-d280083-r926759588-B_Montmartre_Hotel-Paris_Ile_de_France.html
Scraping reviews for hotel: https://www.tripadvisor.in/Hotel_Review-g187147-d276923-Reviews-Best_Western_Plus_La_Demeure-Paris_Ile_de_France.html
Scraping reviews for hotel: https://www.tripadvisor.in/Hotel_Review-g187147-d276923-Reviews-Best_Western_Plus_La_Demeure-Paris_Ile_de_France.html
Scraping reviews for hotel: https://www.tripadvisor.in/Hotel_Review-g187147-d276923-Reviews-Best_Western_Plus_La_Demeure-Paris_Ile_de_France.html#REVIEWS
Scraping reviews for hotel: https://www.tripadvisor.in/ShowUserReviews-g187147-d276923-r887276442-Best_Western_Plus_La_Demeure-Paris_Ile_de_France.html
Empty DataFrame
Columns: []
Index: []


In [56]:
print(len(df))

280


In [58]:
df.to_csv(r'C:\Users\Shivani\OneDrive\Documents\TA_Reviews3.csv',index= False)